**Задача классификации с последующей регрессией. Кол-во таргетов для класс-ии и регресии: 8**

**Если бинарная метка предсказана как 1 - то находим число к этому таргету при помощи регрессионной модели; если бинарная метка 0 - то и регрессионная будет = 0**

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, f1_score
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pickle

In [2]:
X_df = pd.read_csv('X_uuid.csv')
X_df.drop(columns={'Unnamed: 0', 'department_name', 'user_name', 'file_name'}, inplace=True)
X_df.drop(columns=X_df.columns[8:15], inplace=True)
cols = X_df.columns
X_df = X_df.fillna(0)

In [3]:
X = (X_df[X_df.columns[8:]]).to_numpy()
y = (X_df[X_df.columns[:8]]).to_numpy()
y_columns = list(X_df.columns[8:15])

In [ ]:
n_splits = 5 
kf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

bad_f1 = 0
best_model = None

for train_index, val_index in kf.split(X, y):
    f1_values = []

    X_train, X_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]

    forest = RandomForestClassifier(n_estimators=1050, random_state=42, max_depth=13, class_weight='balanced', min_samples_leaf=4, min_samples_split=8, max_features=0.9, oob_score=True)
    multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
    multi_target_forest.fit(X_train, y_train)

    y_pred = multi_target_forest.predict(X_test)

    for i, col in enumerate(y_columns):                                                     # Метрика для каждого столбца таргетов
        val = f1_score(y_test[:, i], y_pred[:, i], average='weighted', zero_division=0)
        f1_values.append(val)

    res_f1 = min(f1_values)                   # Нужно увеличивать минимальный f1 - для этого находим самый плохой f1 из всех колонок

    if(res_f1 > bad_f1):
        bad_f1 = res_f1
        best_model = multi_target_forest
    print(bad_f1)

In [ ]:
bad_f1

In [40]:
with open("Сlassificator.pkl", "wb") as f:
    pickle.dump(best_model, f)

# For Regression

In [41]:
forest_df = pd.read_csv('X_uuid.csv')
forest_df.drop(columns={'Unnamed: 0', 'department_name', 'user_name', 'file_name'}, inplace=True)
forest_df = forest_df.fillna(0)

In [ ]:
X = forest_df[list(forest_df.columns[:8]) + list(forest_df.columns[-3:])].to_numpy()
y = forest_df[forest_df.columns[8:15]].to_numpy()

['scaler_reg.pkl']

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

bad_mae = 10000000000
best_model_reg = None

for train_index, val_index in kf.split(X):
    mae_values = []

    X_train, X_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]

    Regress_forest = RandomForestRegressor(n_estimators=1250, random_state=42, max_depth=14, min_samples_leaf=5, max_features=0.75, min_samples_split=8, oob_score=True)
    Regress = MultiOutputRegressor(Regress_forest, n_jobs=-1)
    Regress.fit(X_train, y_train)

    RG_predict = Regress.predict(X_test)

    for i in range(len(X_test)):                                    # Проходимся по каждому столбцу с таргетом
        mae = mean_absolute_error(y_test[i], RG_predict[i])
        mae_values.append(mae)

    if(max(mae_values) < bad_mae):                                      # Ищем наибольшую ошибку MAE из всех столбцов таргетов, чтобы на следующих итерациях ее минимизировать
        bad_mae = max(mae_values)
        best_model_reg = Regress
    print(bad_mae)

In [50]:
with open("Regresor.pkl", "wb") as f:
    pickle.dump(best_model_reg, f)